# Projection

### Make some imports

In [ ]:
%matplotlib widget
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from pathlib import Path

# from scipy import ndimage as nd

import flopy
print("flopy path: {}".format(flopy.__path__))
print("flopy version: {}".format(flopy.__version__))
import pyemu

### also import some external scripts (where most of the work is done!)

In [ ]:
import sys
sys.path.append("..")
from scripts import utils
from scripts import build

# Attach SLR scenario (e.g., SSP5)

#### We attach the posterior ensemble (fit=6) and attach SLR scenario using 'attach_scenario' function in build.py script 

In [ ]:
t_d = "template_proj_101"
scen = "SSP3"
#m_d = f"master_{scen}"
pred_m = build.attach_scenario(model=t_d, scen=scen) # attach SLR scenario

# Run projection model ensemble 
## posterior Monte Carlo using PEST++ (combine posterior ensemble of parameters for history period with unconditioned temporal parameters)

In [ ]:
nreals=100  # number of realisations for predictive ensemble (if less than nreals in historymatching)
fit=2  # iteration number
nworkers=5 #os.cpu_count()-1  # number of parallel realisation to run (lower if cpu limited)
m_d = f"master_{scen}_100"
utils.prep_and_run("Dunedin_Pred_base.pst", t_d=t_d, m_d=m_d,
                    nreals=nreals, noptmax=-1, nworker=nworkers,
                    prior_pe="prior_pe.jcb",
                    post_pe=os.path.join("..", "master",
                                         f"Dunedin_SS_base.{fit}.par.jcb"))

# Read and process ensemble outputs for projection period

In [ ]:
#pred_pst = "Dunedin_Pred_base.pst"
m_d = f"master_{scen}_100"
m_d =os.path.join("..", m_d)
m = flopy.modflow.Modflow.load(
    f="SouthDun_100.nam", 
    model_ws=m_d, 
    version='mfnwt',
    exe_name='mfnwt.exe', 
    verbose=False, 
    check=True
)
# PROCESS SWEEP OUTPUTS
# re-read pst control file from master dir
try:
    pred_pst.filename = Path(pred_pst.filename)
    pred_pst = pyemu.Pst(os.path.join(m_d, pred_pst.filename.name))
except (AttributeError, NameError):
    pred_pst = "Dunedin_Pred_base.pst"
    pred_pst = pyemu.Pst(os.path.join(m_d, pred_pst))
pstfnme = Path(pred_pst.filename)

#### Read in projection results

In [ ]:
# re ensemble outputs
oe_po_tr = utils.try_load_ensemble(pred_pst, os.path.join(m_d, "Dunedin_Pred_base.0.obs.jcb"), kind='obs')

### Let's look at mapping the probability of groundwater inundation for a specific year of the scenario (e.g., 2100)

In [ ]:
# get pest obs data
obs = pred_pst.observation_data

yr = 2110 # specify year of scenario
kper = yr - 2010

obs = obs.loc[obs.index.str.contains(f'kper:{kper}')]

# just hd outputs
hdobs = obs.loc[obs.obgnme == "oname:hd_otype:lst_usecol:obsval"].astype({c:int for c in ['kper','k','i','j']})
# add column that aligns model top info to hd output names
hdobs['top'] = m.dis.top.array[tuple(hdobs[['i','j']].T.values)]

# calc probabilities of exceed for every output
# Transpose obs ensemble (ensemble outputs), slice for just hd obs, 
# substract model top from every realisation,
# where positive simulated head exceeds model top, count reals where positive, divide by nreal
hdobs['prob'] = ((oe_po_tr.T.loc[hdobs.index].sub(hdobs.top, axis=0) + 0) > 0).sum(axis=1)/oe_po_tr.shape[0]

# create an array from these obs -- WILL NEED TO BE DIF IF MULTPLE KPER AND LAYERS
ar = np.zeros((m.nrow, m.ncol)) # blank array

# add elements from dataframe
ar[tuple(hdobs[['i','j']].T.values)] = hdobs.prob

plt.figure(figsize=(6,6))
plt.title(yr, fontsize=12)
plt.imshow(np.ma.masked_where(m.bas6.ibound.array[0]==0, ar), cmap='plasma')
plt.colorbar()

### Projection of groundwater levels at observation locations

In [ ]:
# get pest obs data
obs = pred_pst.observation_data
#obs.loc[obs.oname=='wl']
w_obs = obs.loc[obs.index.str.contains('sitename')]
sitegroups = w_obs.groupby("sitename")
gps = sitegroups.first().astype({'i':int, 'j':int})
for site, w_obs_df in sitegroups:
    top = m.dis.top.array[int(w_obs_df.i[0]), int(w_obs_df.j[0])]
    at0 = w_obs_df.loc[w_obs_df.kper=='0'].index[0] #2010
    at20 = w_obs_df.loc[w_obs_df.kper=='20'].index[0] #2030
    at40 = w_obs_df.loc[w_obs_df.kper=='40'].index[0] #2050
    at70 = w_obs_df.loc[w_obs_df.kper=='60'].index[0] #2070
    at90 = w_obs_df.loc[w_obs_df.kper=='90'].index[0] #2100
    at100 = w_obs_df.loc[w_obs_df.kper=='100'].index[0] #2100
    
    # Posterior sim out
    tp_0 = oe_po_tr.loc[:, at0]
    tp_20 = oe_po_tr.loc[:, at20]  
    tp_40 = oe_po_tr.loc[:, at40]
    #tp_60 = oe_po_tr.loc[:, at60]
    tp_70 = oe_po_tr.loc[:, at70]
    tp_90 = oe_po_tr.loc[:, at90]

    tp_d_40 = tp_40 - tp_0
    tp_d_20 = tp_20 - tp_0
    #tp_d_60 = tp_60 - tp_0
    tp_d_70 = tp_70 - tp_0
    tp_d_90 = tp_90 - tp_0
    
    fig, axes = plt.subplots(1,3, figsize=(12,4))
    ax = axes[0]
    ax.imshow(np.ma.masked_where(m.bas6.ibound.array[0] == 0, ar),
               cmap="plasma", interpolation='none')
    ax.scatter(*gps.loc[site, ['j', 'i']].values, marker='v', color='r')
    ax.scatter(*gps.loc[site, ['j', 'i']].values, marker='o', color='w', fc='none', s=120, lw=2)
    
    ax = axes[1]
    tp_d_90.hist(ax=ax, bins=25, color='r', density=False, label='2100', alpha=0.6)
    tp_d_70.hist(ax=ax, bins=25, color='y', density=False, label='2070', alpha=0.8)
    tp_d_40.hist(ax=ax, bins=25, color='b', density=False, label='2050', alpha=0.8)
    tp_d_20.hist(ax=ax, bins=25, color='grey', density=False, label='2030', alpha=0.8)
    #tp_d_70.hist(ax=ax, bins=25, color='k', alpha=0.5, density=False)
    ax.set_title(site, fontsize=9)
    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=7)
    ax.set_xlabel('gw level change (m)', fontsize=9)
    ax.set_ylabel('frequency', fontsize=9)
    ys = ax.get_ylim()
    ax.plot((0.25,0.25), ys, color='k', ls='--', label='Model top')
    ax.plot((0.25,0.25), ys, color='r', ls='--', label='Decision threshold')
    legend = ax.legend(loc='best', fontsize=7)
                  
    ax = axes[2]            
    #oe_rcp26.loc[:, w_obs_df.astype({'kper':int}).sort_values('kper').index].T.reset_index().plot(ax=ax, legend=False, color='green', alpha=0.2)
    oe_po_tr.loc[:, w_obs_df.astype({'kper':int}).sort_values('kper').index].T.reset_index().plot(ax=ax, legend=False, color='r', alpha=0.2)
    #oe_rcp85.loc[:, w_obs_df.astype({'kper':int}).sort_values('kper').index].T.reset_index().plot(ax=ax, legend=False, color='red', alpha=0.2)
    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=7)
    ax.set_xlabel('Years from start of projection', fontsize=9)
    ax.set_ylabel('gw level (m OMD)', fontsize=9)
    ax.set_title(site, fontsize=9)
    xs = ax.get_xlim()   
    ax.plot(xs, (top,top), color='k', ls='--', label='model top')
    ax.set_xlim(0,100)
        
    #plt.savefig("plots/SSP5/SSP5_po_{0}.png".format(site))

### Predictions for total drain flux -- over time

In [ ]:
#get pest obs data
obs = pred_pst.observation_data
drnsumobs = obs.loc[obs.index.str.startswith('oname:drnsum')] #.astype({c:int for c in ['kper','k','i','j']})
# Posterior sim out
drnsumoe_df = oe_po_tr.loc[:, drnsumobs.index]
drnsumoe_df_pos = drnsumoe_df * -1

at_0 = drnsumoe_df.loc[:, 'oname:drnsum_otype:lst_usecol:sum_kper:0']
at_20 = drnsumoe_df.loc[:, 'oname:drnsum_otype:lst_usecol:sum_kper:20']
at_40 = drnsumoe_df.loc[:, 'oname:drnsum_otype:lst_usecol:sum_kper:40']
at_60 = drnsumoe_df.loc[:, 'oname:drnsum_otype:lst_usecol:sum_kper:60']
at_90 = drnsumoe_df.loc[:, 'oname:drnsum_otype:lst_usecol:sum_kper:90']

drn_0 = at_0 * -1
drn_20 = at_20 * -1
drn_40 = at_40 * -1
drn_60 = at_60 * -1
drn_90 = at_90 * -1

diff_20 = at_20 - at_0
diff_40 = at_40 - at_0
diff_60 = at_60 - at_0
diff_90 = at_90 - at_0

fig, axes = plt.subplots(1,2, figsize=(12,4))

ax = axes[0]
drnsumoe_df_pos.loc[:, drnsumobs.astype({'kper':int}).sort_values('kper').index].T.reset_index().plot(ax=ax, legend=False, color='red', alpha=0.2)
ax.set_title("Drain flux versus time", fontsize=10)
ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=10)
ax.set_xlim(0,100)
#ax.set_ylim(-6000, 0)
ax.set_xlabel('year of scenario', fontsize=10)
ax.set_ylabel('flux m$^3$/day', fontsize=10)

ax = axes[1]
drn_90.hist(ax=ax, bins=25, color='r', alpha=0.5, density=False, label='2100')
drn_60.hist(ax=ax, bins=25, color='y', alpha=0.8, density=False, label='2070')
drn_40.hist(ax=ax, bins=25, color='b', alpha=0.8, density=False, label='2050')
drn_20.hist(ax=ax, bins=25, color='0.5', alpha=0.9, density=False, label='2030')
ax.set_title("Drain flux PDF", fontsize=10)
ax.tick_params(axis='x', labelsize=10)
ax.tick_params(axis='y', labelsize=10)
ax.set_xlabel('flux m$^3$/day', fontsize=10)
ax.set_ylabel('frequency', fontsize=10)
legend = ax.legend(loc='best', fontsize=10)
#plt.savefig("plots/SSP5/drain_flux_versus_time.pdf")
#ax.set_ylim(0, 30)

__________
## Optional save of ensemble output for downstream vis

In [ ]:
from scripts.utils import extract_hdobs_to_pkl, extract_hdoe_to_pkl
hdobs = extract_hdobs_to_pkl(pred_pst)
extract_hdoe_to_pkl(oe_po_tr, hdobs, pred_pst)

In [ ]:
viewer_d = Path('..','..','searise_viewer_sd')
Path(viewer_d, 'scenarios', scen).mkdir(parents=True, exist_ok=True)
shutil.copy(Path(m_d, "hdoe.pkl"), Path(viewer_d, 'scenarios', scen))
Path(viewer_d, 'model').mkdir(parents=True, exist_ok=True)
pred_pst.write(Path(viewer_d, "model", Path(pred_pst.filename).name))